In [1]:
import pandas as pd
import h5py
file_name='../data/WIKI_PRICES_212b326a081eacca455e13140d7bb9db.csv'
stock_path = '../data/stock_prices.h5'

us_stocks=pd.read_csv(file_name, parse_dates=['date']).set_index(['date'])  # Load data
us_stocks=us_stocks.loc['2000-1-1':].reset_index().set_index(['date', 'ticker'])  # Keep after 2000

# Simplficiation
us_stocks.drop(columns=list(us_stocks.filter(regex='^(?!adj_)')), inplace=True)
us_stocks.columns=[i.replace('adj_','') for i in list(us_stocks)]
us_stocks.sample(5)

,,open,high,low,close,volume
date,ticker,,,,,
2006-04-18,SPA,8.500000,8.500000,8.500000,8.500000,200.0
2011-01-14,HBIO,2.532376,2.538251,2.502998,2.526500,9700.0
2009-04-14,DFZ,4.561387,5.226590,4.561387,4.889669,20300.0
2013-01-15,FRNK,17.010000,17.120000,17.000000,17.100000,6600.0
2016-12-14,RNST,41.071623,41.298647,40.420162,40.903822,1240690.0


In [21]:
# Write simplified version as HDF5 file
store = pd.HDFStore(stock_path)

In [22]:
# us_stocks.loc[us_stocks.index.get_level_values(0).get_loc('2001')]
dates={
    'index':us_stocks.index.get_level_values(0)
    ,'training': ('2006-1-1', '2013-12-31')
    ,'testing': ('2014-1-1', '2016-12-31')
    ,'validation': ('2017-1-1', '2018-12-31')
}
masks = {
    'pre_training':dates['index']< dates['training'][0],
    'training':   (dates['index']>=dates['training'][0])   & (dates['index']< dates['testing'][0]),
    'testing':    (dates['index']>=dates['testing'][0])    & (dates['index']< dates['validation'][0]),
    'validation': (dates['index']>=dates['validation'][0]) & (dates['index']<=dates['validation'][1])
}


In [23]:
us_stocks[masks['pre_training']]
store['/data/pre_training'] = us_stocks[masks['pre_training']]
store['/data/training'] = us_stocks[masks['training']]
store['/data/testing'] = us_stocks[masks['testing']]
store['/data/validation'] = us_stocks[masks['validation']]

In [24]:
store.get('/data/validation').sample(5)

,,open,high,low,close,volume
date,ticker,,,,,
2017-03-28,VRSK,79.390000,80.37000,79.240000,80.060000,592919.0
2017-03-13,NRG,17.795631,17.98484,17.198128,17.257878,9000158.0
2018-02-21,PFPT,105.820000,106.84000,104.840000,105.100000,726526.0
2018-03-09,HIBB,23.250000,23.62500,22.650000,22.850000,721371.0
2017-02-27,XCRA,8.980000,9.11000,8.930000,9.020000,355707.0


In [25]:
store.close()